In [34]:
!git clone https://github.com/renatasuyudi/kapsel.git

fatal: destination path 'kapsel' already exists and is not an empty directory.


In [35]:
#from google.colab import drive
#drive.mount('/content/drive')

In [36]:
import pandas as pd

In [37]:
data1 = pd.read_json('/content/kapsel/data/dataPCR.json', lines=True)
data2 = pd.read_json('/content/kapsel/data/datacovid.json', lines=True)
data3 = pd.read_json('/content/kapsel/data/datacovid19.json', lines=True)
data4 = pd.read_json('/content/kapsel/data/dataisoman.json', lines=True)
data5 = pd.read_json('/content/kapsel/data/datakarantina.json', lines=True)
data6 = pd.read_json('/content/kapsel/data/datapandemicovid.json', lines=True)
data7 = pd.read_json('/content/kapsel/data/datappkm.json', lines=True)
data8 = pd.read_json('/content/kapsel/data/datapsbb.json', lines=True)
data9 = pd.read_json('/content/kapsel/data/dataswab.json', lines=True)
data=pd.concat([data1,data2,data3,data4,data5,data6,data7,data8,data9]).reset_index(drop=True)

In [38]:
data

,created_at,id,id_str,full_text,truncated,display_text_range,entities,metadata,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user,geo,coordinates,place,contributors,retweeted_status,is_quote_status,retweet_count,favorite_count,favorited,retweeted,lang,quoted_status_id,quoted_status_id_str,quoted_status,possibly_sensitive,extended_entities
0,2021-10-30 14:04:48+00:00,1454449208606429184,1454449208606429184,RT @perupadata: Resmi: harga tes RT-PCR turun....,False,"[0, 139]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'in', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,NaN,NaN,NaN,None,"{'id': 105504840, 'id_str': '105504840', 'name...",NaN,NaN,None,NaN,{'created_at': 'Wed Oct 27 10:28:23 +0000 2021...,False,283,0,False,False,in,NaN,NaN,NaN,NaN,NaN
1,2021-10-30 14:04:27+00:00,1454449121989824519,1454449121989824512,RT @detikfinance: #mostpop Mantan Sekretaris M...,False,"[0, 139]","{'hashtags': [{'text': 'mostpop', 'indices': [...","{'iso_language_code': 'in', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,NaN,NaN,NaN,None,"{'id': 1019403817295605760, 'id_str': '1019403...",NaN,NaN,None,NaN,{'created_at': 'Sat Oct 30 13:48:14 +0000 2021...,False,1,0,False,False,in,NaN,NaN,NaN,NaN,NaN
2,2021-10-30 14:04:26+00:00,1454449116356907016,1454449116356907008,@mindweul Biar bersatu dgn pcr gepengmu,False,"[10, 39]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'in', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",1.454449e+18,1.454449e+18,1.451753e+18,1.451753e+18,mindweul,"{'id': 1160690231865008128, 'id_str': '1160690...",NaN,NaN,None,NaN,NaN,False,0,0,False,False,in,NaN,NaN,NaN,NaN,NaN
3,2021-10-30 14:04:25+00:00,1454449111919325189,1454449111919325184,RT @ManassaDaeng: Pemerimtah sudah menurunkan ...,False,"[0, 140]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'in', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,NaN,NaN,NaN,None,"{'id': 1192569236494749696, 'id_str': '1192569...",NaN,NaN,None,NaN,{'created_at': 'Sat Oct 30 01:33:28 +0000 2021...,False,40,0,False,False,in,NaN,NaN,NaN,NaN,NaN
4,2021-10-30 14:04:12+00:00,1454449059004092417,1454449059004092416,@2pec__ agr pcr,False,"[8, 15]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'in', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",1.454449e+18,1.454449e+18,7.982780e+17,7.982780e+17,2pec__,"{'id': 1107842529590951936, 'id_str': '1107842...",NaN,NaN,None,NaN,NaN,False,0,0,False,False,in,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30098,2021-10-27 03:38:19+00:00,1453204387006857223,1453204387006857216,"Antisipasi Gelombang III Covid-19, BNN Prov Su...",False,"[0, 273]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'in', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,NaN,NaN,NaN,None,"{'id': 2923492080, 'id_str': '2923492080', 'na...",None,None,None,NaN,NaN,False,0,0,False,False,in,NaN,NaN,NaN,0.0,"{'media': [{'id': 1453203835674005507, 'id_str..."
30099,2021-10-27 03:38:18+00:00,1453204381885554691,1453204381885554688,@Hilmi28 dan saya ngalamin 😑\nswab antigen 170...,False,"[9, 151]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'in', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",1.452798e+18,1.452798e+18,6.506718e+07,6.506718e+07,Hilmi28,"{'id': 1120730554645630976, 'id_str': '1120730...",None,None,None,NaN,NaN,False,0,1,False,False,in,NaN,NaN,NaN,NaN,NaN
30100,2021-10-27 03:36:42+00:00,1453203977349177348,1453203977349177344,"mana tadi idung aing di rojok dalem pisan, ya ...",False,"[0, 62]","{'hashtags

In [39]:
pip install Sastrawi

In [40]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [41]:
# Loading Stopwords: Ada beberapa cara
from nltk.corpus import stopwords
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
factory = StopWordRemoverFactory()

NLTK_StopWords = stopwords.words('english')
Sastrawi_StopWords_id = factory.get_stop_words()

df=open('/content/kapsel/stopword_en/stopword_en.txt',"r",encoding="utf-8", errors='replace')
en_stop = df.readlines()
df.close()
en_stop = [t.strip().lower() for t in en_stop]

df=open('/content/kapsel/stopword/stopword.txt',"r",encoding="utf-8", errors='replace')
id_stop = df.readlines()
df.close()
id_stop = [t.strip().lower() for t in id_stop]

N = 10
print(NLTK_StopWords[:N])
print(Sastrawi_StopWords_id[:N])
print(en_stop[:N])
print(id_stop[:N])
print(len(Sastrawi_StopWords_id), len(id_stop), len(NLTK_StopWords), len(en_stop))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]
['yang', 'untuk', 'pada', 'ke', 'para', 'namun', 'menurut', 'antara', 'dia', 'dua']
['&gt', '&lt', '&nbsp', 'a', 'able', 'about', 'above', 'abst', 'accordance', 'according']
['ada', 'adalah', 'adanya', 'adapun', 'agak', 'agaknya', 'agar', 'akan', 'akankah', 'akhir']
126 758 179 2659


In [42]:
datafinal=data[['full_text']]

In [43]:
datafinal

,full_text
0,RT @perupadata: Resmi: harga tes RT-PCR turun....
1,RT @detikfinance: #mostpop Mantan Sekretaris M...
2,@mindweul Biar bersatu dgn pcr gepengmu
3,RT @ManassaDaeng: Pemerimtah sudah menurunkan ...
4,@2pec__ agr pcr
...,...
30098,"Antisipasi Gelombang III Covid-19, BNN Prov Su..."
30099,@Hilmi28 dan saya ngalamin 😑\nswab antigen 170...
30100,"mana tadi idung aing di rojok dalem pisan, ya ..."
30101,ples td di sekolah tbtb swab dadakan


In [44]:
pip install docx2txt

In [45]:
pip install unidecode

In [46]:
import re, os, itertools, docx2txt
from tqdm import tqdm
#from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk import sent_tokenize, word_tokenize
from spacy.lang.id import Indonesian
from html import unescape
from unidecode import unidecode
import pandas as pd
#from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
#from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.tokenize import TweetTokenizer; Tokenizer = TweetTokenizer(reduce_len=True)
#from nltk.corpus import wordnet as wn
from nltk.stem import PorterStemmer;ps = PorterStemmer()
from string import punctuation
#from pattern.web import PDF
from bz2 import BZ2File as bz2
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA
import matplotlib.pyplot as plt
import numpy as np
from textblob import TextBlob 
#from NLP_Models import openewfile as of

In [47]:
def crawlFiles(dPath, types = None):
    #dPath = 'C:/Temp', types ='pdf'
    if types:
        return [dPath+'/'+f for f in os.listdir(dPath) if f.endswith('.'+types)]
    else:
        return [dPath+'/'+f for f in os.listdir(dPath)]

In [48]:
def readBz2(file):
    with bz2(file, "r") as bzData:
        txt = []
        for line in bzData:
            try:
                txt.append(line.strip().decode('utf-8','replace'))
            except:
                pass
    return ' '.join(txt)

In [49]:
def LoadDocuments(dPath=None,types=None, file = None): # types = ['pdf','doc','docx','txt','bz2']
    Files, Docs = [], []
    if types:
        for tipe in types:
            Files += crawlFiles(dPath,tipe)
    if file:
        Files = [file]
    if not types and not file: # get all files regardless of their extensions
        Files += crawlFiles(dPath)
    for f in Files:
#        if f[-3:].lower()=='pdf':
#            try:
#                Docs.append(PDF(f).string)
#            except:
#                print('error reading{0}'.format(f))
        if f[-3:].lower()=='txt':
            try:
                df=open(f,"r",encoding="utf-8", errors='replace')
                Docs.append(df.readlines());df.close()
            except:
                print('error reading{0}'.format(f))
        elif f[-3:].lower()=='bz2':
            try:
                Docs.append(readBz2(f))
            except:
                print('error reading{0}'.format(f))
        elif f[-4:].lower()=='docx':
            try:
                Docs.append(docx2txt.process(f))
            except:
                print('error reading{0}'.format(f))
        elif f[-3:].lower()=='csv':
            Docs.append(pd.read_csv(f))
        else:
            print('Unsupported format {0}'.format(f))
    if file:
        Docs = Docs
    return Docs, Files

In [50]:
def LoadStopWords(lang, sentiment = True):
    L = lang.lower().strip()
    if sentiment:
        if L == 'en' or L == 'english' or L == 'inggris':
            lemmatizer = WordNetLemmatizer()
            stops = set([t.strip() for t in LoadDocuments(file = '/content/kapsel/stopword')[0]])
        elif L == 'id' or L == 'indonesia' or L == 'indonesian':
            lemmatizer = Indonesian()
            stops = set([t.strip() for t in LoadDocuments(file ='/content/kapsel/stopword_noise')[0]])
        else:
            print('Warning! Languange not recognized. Empty stopword given')
            stops = set(); lemmatizer = None
    else:
        if L == 'en' or L == 'english' or L == 'inggris':
            lemmatizer = WordNetLemmatizer()
            stops = set([t.strip() for t in LoadDocuments(file = '/content/kapsel/stopword_en')[0]])
        elif L == 'id' or L == 'indonesia' or L == 'indonesian':
            lemmatizer = Indonesian()
            stops = set([t.strip() for t in LoadDocuments(file = '/content/kapsel/stopword')[0]])
        else:
            print('Warning! Languange not recognized. Empty stopword given')
            stops = set(); lemmatizer = None
    return stops, lemmatizer

In [51]:
def fixTags(T):
    getHashtags = re.compile(r"#(\w+)")
    pisahtags = re.compile(r'[A-Z][^A-Z]*')
    t = T
    tagS = re.findall(getHashtags, T)
    for tag in tagS:
        proper_words = ' '.join(re.findall(pisahtags, tagS[0]))
        t = t.replace('#'+tag, proper_words)
    return t

In [52]:
def getTags(T):
    getHashtags = re.compile(r"#(\w+)")
    tagS = re.findall(getHashtags, T)
    isitag = []
    for tag in tagS:       
        tag = '#'+tag
        isitag.append(tag)
    
    return ', '.join(isitag)

In [53]:
def cleanText(T, fix={}, pattern2 = False, lang = 'id', lemma=None, stops = set(), symbols_remove = False, numbers_remove = False, hashtag_remove = False, min_charLen = 0):
    # lang & stopS only 2 options : 'en' atau 'id'
    # symbols ASCII atau alnum
    pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    pattern1 = re.compile(r'pic.twitter.com/(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    if pattern2:
        pattern2 = re.compile(r'@(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+') #remove@
        t = re.sub(pattern2, ' ',T)
    else:
        t = T
    t = re.sub(pattern,' ',t) #remove urls if any
    t = re.sub(pattern1,' ',t)
    t = unescape(t) # html entities fix
    if hashtag_remove:
        t = fixTags(t) # fix abcDef
    else:
        t = t
    t = t.lower().strip() # lowercase
    t = unidecode(t)
    t = ''.join(''.join(s)[:2] for _, s in itertools.groupby(t)) # remove repetition
    t = sent_tokenize(t) # sentence segmentation. String to list
    for i, K in enumerate(t):
        if symbols_remove:
            K = re.sub(r'[^.,a-zA-Z0-9 \n\.]',' ',K)
            K = K.replace(',',' ').replace('.',' ')
            K = ''.join(c for c in K if c not in punctuation)
            K = re.sub('\s+',' ',K).strip()
        
        if numbers_remove:
            K = re.sub(r'[0-9]',' ',K)
            K = re.sub('\s+',' ',K)
            
        cleanList = []
        if lang =='en':
            listKata = word_tokenize(K) # word tokenize
            for token in listKata:
                if token in fix.keys():
                    token = fix[token]
                if lemma:
                    token = lemma.lemmatize(token)
                if stops:
                    if len(token)>=min_charLen and token not in stops:
                        cleanList.append(token)
                else:
                    if len(token)>=min_charLen:
                        cleanList.append(token)
            t[i] = ' '.join(cleanList)
        else:
            if lemma:
                K = lemma(K)
                listKata = [token.text for token in K]
            else:
                listKata = TextBlob(K).words
                
            for token in listKata:
                if token in fix.keys():
                    token = fix[token]
                
                if lemma:
                    token = lemma(token)[0].lemma_
                    #token = stemmer.stem(token)
                if stops:    
                    if len(token)>=min_charLen and token not in stops:
                        cleanList.append(token)
                else:
                    if len(token)>=min_charLen:
                        cleanList.append(token)
            t[i] = ' '.join(cleanList).lstrip()
    return ' '.join(t) # Return kalimat lagi


In [54]:
def cleanText_(T, fix={}, min_charLen = 0):
    t = T
    t = t.lower().strip() # lowercase
    t = unidecode(t)
    t = ''.join(''.join(s)[:2] for _, s in itertools.groupby(t)) # remove repetition
    t = sent_tokenize(t) # sentence segmentation. String to list
    for i, K in enumerate(t):
        cleanList = []
        listKata = TextBlob(K).words
        for token in listKata:
            if token in fix.keys():
                token = fix[token]
                cleanList.append(token)
            else:
                if len(token)>=min_charLen:
                    cleanList.append(token)
        t[i] = ' '.join(cleanList).lstrip()
    return ' '.join(t) # Return kalimat lagi


In [55]:
def handlingnegation (full_text):
    match = re.compile(r'(tidak|kurang|bukan|jangan|tapi|tetapi) (\w+)').findall(full_text)
    match = list(set(match))
    for i,word in enumerate(match): 
        if ' '.join(match[i]) in full_text:
            kata = full_text.replace(' '.join(match[i]), str(match[i][0])+' '+'negx'+str(match[i][1]))
            full_text = kata
    return full_text

In [56]:
def handlingporn (full_text):
    match = re.compile(r'(video|foto|photo) (\w+)').findall(full_text)
    match = list(set(match))
    for i,word in enumerate(match): 
        if ' '.join(match[i]) in full_text:
            kata = full_text.replace(' '.join(match[i]), str(match[i][0])+' '+'pornx'+str(match[i][1]))
            full_text = kata
    return full_text

In [57]:
def print_Topics(model, feature_names, Top_Topics, n_top_words):
    for topic_idx, topic in enumerate(model.components_[:Top_Topics]):
        print("Topic #%d:" %(topic_idx+1))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))


In [58]:
def getTopics(Txt,n_topics=5, Top_Words=7):
    #Txt = [t['nlp'] for t in Tweets] # cleaned: stopwords, stemming
    tf_vectorizer = CountVectorizer(strip_accents = 'unicode', token_pattern = r'\b[a-zA-Z]{3,}\b', max_df = 0.95, min_df = 2)
    dtm_tf = tf_vectorizer.fit_transform(Txt)
    tf_terms = tf_vectorizer.get_feature_names()
    lda_tf = LDA(n_components=n_topics, learning_method='online', random_state=0).fit(dtm_tf)   
    vsm_topics = lda_tf.transform(dtm_tf); doc_topic =  [a.argmax()+1 for a in tqdm(vsm_topics)] # topic of docs
    print('In total there are {0} major topics, distributed as follows'.format(len(set(doc_topic))))
    fig4 = plt.figure(); fig4.add_subplot(111)
    plt.hist(np.array(doc_topic), alpha=0.5); plt.show()
    print('Printing top {0} Topics, with top {1} Words:'.format(n_topics, Top_Words))
    print_Topics(lda_tf, tf_terms, n_topics, Top_Words)
    return lda_tf, dtm_tf, tf_vectorizer

In [59]:
#from NLP_Models import TextMining as tm
import time
#from NLP_Models import openewfile as of
from tqdm import tqdm
#import swifter

In [60]:
def cleanningtext(data, both = True, onlyclean = False, sentiment = False):
    print('Cleaning Text')
    fSlang = '/content/kapsel/slangword/slang.txt'
    bahasa = 'id'
    stops, lemmatizer = LoadStopWords(bahasa, sentiment = sentiment)
    sw=open(fSlang,encoding='utf-8', errors ='ignore', mode='r');SlangS=sw.readlines();sw.close()
    SlangS = {slang.strip().split(':')[0]:slang.strip().split(':')[1] for slang in SlangS}
  
    start_time = time.time()
    tqdm.pandas()
    
    if both:
        data['full_text'] = data['full_text'].astype('str')
        data['full_text'] = data['full_text'].str.lower()
        data = data[~data.full_text.str.contains('unavailable')]
        data['cleaned_text'] = data['full_text'].progress_apply(lambda x : cleanText(x,fix=SlangS, pattern2 = True, lang = bahasa, lemma=lemmatizer, stops = stops, symbols_remove = True, numbers_remove = True, hashtag_remove=False, min_charLen = 2))
        data['cleaned_text'] = data['cleaned_text'].progress_apply(lambda x : handlingnegation(x))
        #data['cleaned_text'] = data['cleaned_text'].progress_apply(lambda x : handlingporn(x))
    elif onlyclean: 
        data['cleaned_text'] = data['full_text'].progress_apply(lambda x : cleanText(x, fix=SlangS, pattern2 = True, lang = bahasa, lemma=lemmatizer, stops = stops, symbols_remove = True, numbers_remove = True, hashtag_remove=False, min_charLen = 3))
    else:
        data['cleaned_text'] = data['full_text'].progress_apply(lambda x : handlingnegation(x))
    
    data = data[data['cleaned_text'].notna()]
    print("%s seconds" %(time.time()-start_time))
    
    return data

In [61]:
data

,created_at,id,id_str,full_text,truncated,display_text_range,entities,metadata,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user,geo,coordinates,place,contributors,retweeted_status,is_quote_status,retweet_count,favorite_count,favorited,retweeted,lang,quoted_status_id,quoted_status_id_str,quoted_status,possibly_sensitive,extended_entities
0,2021-10-30 14:04:48+00:00,1454449208606429184,1454449208606429184,RT @perupadata: Resmi: harga tes RT-PCR turun....,False,"[0, 139]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'in', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,NaN,NaN,NaN,None,"{'id': 105504840, 'id_str': '105504840', 'name...",NaN,NaN,None,NaN,{'created_at': 'Wed Oct 27 10:28:23 +0000 2021...,False,283,0,False,False,in,NaN,NaN,NaN,NaN,NaN
1,2021-10-30 14:04:27+00:00,1454449121989824519,1454449121989824512,RT @detikfinance: #mostpop Mantan Sekretaris M...,False,"[0, 139]","{'hashtags': [{'text': 'mostpop', 'indices': [...","{'iso_language_code': 'in', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,NaN,NaN,NaN,None,"{'id': 1019403817295605760, 'id_str': '1019403...",NaN,NaN,None,NaN,{'created_at': 'Sat Oct 30 13:48:14 +0000 2021...,False,1,0,False,False,in,NaN,NaN,NaN,NaN,NaN
2,2021-10-30 14:04:26+00:00,1454449116356907016,1454449116356907008,@mindweul Biar bersatu dgn pcr gepengmu,False,"[10, 39]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'in', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",1.454449e+18,1.454449e+18,1.451753e+18,1.451753e+18,mindweul,"{'id': 1160690231865008128, 'id_str': '1160690...",NaN,NaN,None,NaN,NaN,False,0,0,False,False,in,NaN,NaN,NaN,NaN,NaN
3,2021-10-30 14:04:25+00:00,1454449111919325189,1454449111919325184,RT @ManassaDaeng: Pemerimtah sudah menurunkan ...,False,"[0, 140]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'in', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,NaN,NaN,NaN,None,"{'id': 1192569236494749696, 'id_str': '1192569...",NaN,NaN,None,NaN,{'created_at': 'Sat Oct 30 01:33:28 +0000 2021...,False,40,0,False,False,in,NaN,NaN,NaN,NaN,NaN
4,2021-10-30 14:04:12+00:00,1454449059004092417,1454449059004092416,@2pec__ agr pcr,False,"[8, 15]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'in', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",1.454449e+18,1.454449e+18,7.982780e+17,7.982780e+17,2pec__,"{'id': 1107842529590951936, 'id_str': '1107842...",NaN,NaN,None,NaN,NaN,False,0,0,False,False,in,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30098,2021-10-27 03:38:19+00:00,1453204387006857223,1453204387006857216,"Antisipasi Gelombang III Covid-19, BNN Prov Su...",False,"[0, 273]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'in', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,NaN,NaN,NaN,None,"{'id': 2923492080, 'id_str': '2923492080', 'na...",None,None,None,NaN,NaN,False,0,0,False,False,in,NaN,NaN,NaN,0.0,"{'media': [{'id': 1453203835674005507, 'id_str..."
30099,2021-10-27 03:38:18+00:00,1453204381885554691,1453204381885554688,@Hilmi28 dan saya ngalamin 😑\nswab antigen 170...,False,"[9, 151]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'in', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",1.452798e+18,1.452798e+18,6.506718e+07,6.506718e+07,Hilmi28,"{'id': 1120730554645630976, 'id_str': '1120730...",None,None,None,NaN,NaN,False,0,1,False,False,in,NaN,NaN,NaN,NaN,NaN
30100,2021-10-27 03:36:42+00:00,1453203977349177348,1453203977349177344,"mana tadi idung aing di rojok dalem pisan, ya ...",False,"[0, 62]","{'hashtags

In [62]:
dataakhir = cleanningtext(data=data, both=True, onlyclean=False, sentiment=False)

Cleaning Text
Unsupported format /content/kapsel/stopword


100%|██████████| 30103/30103 [00:00<00:00, 195612.37it/s]


75.82113647460938 seconds


In [63]:
data.keys()

Index(['created_at', 'id', 'id_str', 'full_text', 'truncated',
       'display_text_range', 'entities', 'metadata', 'source',
       'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str',
       'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place',
       'contributors', 'retweeted_status', 'is_quote_status', 'retweet_count',
       'favorite_count', 'favorited', 'retweeted', 'lang', 'quoted_status_id',
       'quoted_status_id_str', 'quoted_status', 'possibly_sensitive',
       'extended_entities'],
      dtype='object')

In [64]:
dataakhir.keys()

Index(['created_at', 'id', 'id_str', 'full_text', 'truncated',
       'display_text_range', 'entities', 'metadata', 'source',
       'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str',
       'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place',
       'contributors', 'retweeted_status', 'is_quote_status', 'retweet_count',
       'favorite_count', 'favorited', 'retweeted', 'lang', 'quoted_status_id',
       'quoted_status_id_str', 'quoted_status', 'possibly_sensitive',
       'extended_entities', 'cleaned_text'],
      dtype='object')

In [65]:
voyant = dataakhir[['cleaned_text']]

In [66]:
voyant

,cleaned_text
0,rt resmi harga tes rt pcr turun pemerintah men...
1,rt mostpop mantan sekretaris menteri bumn muha...
2,biar bersatu dengan pcr gepengmu
3,rt pemerimtah sudah menurunkan tes pcr menjadi...
4,agr pcr
...,...
30098,antisipasi gelombang ii covid bnn prov sumbar ...
30099,dan saya ngalamin swab antigen rb positif kela...
30100,mana tadi idung aing di rojok dalem pisan ya k...
30101,ples td di sekolah tbtb swab dadakan


In [67]:
voyant.to_csv('/content/kapsel/data/voyant.txt', index=False)

In [68]:
dataoke = dataakhir[['full_text', 'cleaned_text']]

In [69]:
dataoke.full_text[0]

'rt @perupadata: resmi: harga tes rt-pcr turun. pemerintah mengumumkan sore ini.\n\ndi jawa-bali, harga tes pcr maksimal rp275.000, sementara…'

In [70]:
dataoke.cleaned_text[0]

'rt resmi harga tes rt pcr turun pemerintah mengumumkan sore ini di jawa bali harga tes pcr maksimal rp sementara'